[E-06]lyrics_generation
* 목적 : 시퀀스 데이터 대상으로 딥러닝 모델을 학습 하여 다음 시퀀스를 생성해보자. 
* 활용데이터 : 가사의 시퀀스 형태 데이터(8:2)
* 과정 : 시퀀스 데이터의 정제 테크닉을 활용하여 토큰화, 특수 문자 제거등 전처리를 진행하고 RNN 계열 딥러닝 모델을 활용하여 다음 노래 가사의 텍스트를 생성한다.
* 목표 : 텍스트 생성 모델의 10epoch 이내에 validation loss 2.2 이하/ 생성결과의 해석가능 여부

---

1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가?
    - 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가?
2. 가사 텍스트 생성 모델이 정상적으로 작동하는가?
    - 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?
3. 텍스트 생성모델이 안정적으로 학습되었는가?
    - 텍스트 생성모델의 validation loss가 2.2이하로 낮아졌는가?
---

# 0. 필요 module import

In [18]:
import os
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf 
import re

print(tf.__version__)

2.6.0


# 1. 데이터 다운로드

In [ ]:
# !ln -s ~/data ~/aiffel/lyricist/data

# 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' 

txt_list = glob.glob(txt_file_path) 

raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() 
        raw_corpus.extend(raw) 

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


# 3. 데이터 정제
1. 모든 문자를 소문자로. 공백 제거 
2. 특수문자 양쪽에 공백 추가
3. 공백 패턴을 모두 하나의 공백으로 변환
4. a-zA-Z.!,? 해당 문자 제외 문자는 공백으로 변환
5. 문장 양끝 공백 제거
6. 문장 시작과 끝 표시  
7. 토큰화 : 최대 토큰갯수 15개(권장사항)

In [3]:
def preprocess_sentence(sentence): 
    sentence = sentence.lower().strip() # 1 : 모든 문자의 소문자 처리와 공백제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 : 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 3 : 공백 패턴은 스페이스로 변환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 : 해당 문자를 제외한 문자는(공백포함) 스페이스로 변환
    sentence = sentence.strip() # 5 : 문장 양끝 공백제거
    sentence = '<start> ' + sentence + ' <end>' # 6 : 문장 앞뒤 표시
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:

    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 전체 단어의 수 (12000: 권장)
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[   2 2701 2584 ...    0    0    0]
 [   2    7  156 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2  311    1 ...    0    0    0]
 [   2  735    5 ...    0    0    0]
 [   2  735    5 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f6827e5d8b0>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print('src_input', src_input.shape)
print(src_input[0])
print('tgt_input' , tgt_input.shape)
print(tgt_input[0])

src_input (175749, 19)
[   2 2701 2584    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
tgt_input (175749, 19)
[2701 2584    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


# 4. 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size=0.2, 
                                                          shuffle=True)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

# 5. 인공지능 만들기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.


In [11]:
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                     reduction='none') 

model.compile(loss=loss, optimizer=optimizer)     
history = model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data=(enc_val, dec_val), verbose=2)
                    

Epoch 1/10
550/550 - 137s - loss: 3.0349 - val_loss: 2.6907
Epoch 2/10
550/550 - 131s - loss: 2.5685 - val_loss: 2.4906
Epoch 3/10
550/550 - 131s - loss: 2.4091 - val_loss: 2.3788
Epoch 4/10
550/550 - 131s - loss: 2.2953 - val_loss: 2.2999
Epoch 5/10
550/550 - 131s - loss: 2.2011 - val_loss: 2.2395
Epoch 6/10
550/550 - 131s - loss: 2.1159 - val_loss: 2.1868
Epoch 7/10
550/550 - 131s - loss: 2.0355 - val_loss: 2.1405
Epoch 8/10
550/550 - 131s - loss: 1.9593 - val_loss: 2.1040
Epoch 9/10
550/550 - 131s - loss: 1.8880 - val_loss: 2.0730
Epoch 10/10
550/550 - 140s - loss: 1.8206 - val_loss: 2.0473


## 학습된 모델 결과 확인하기

In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated 

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)


'<start> i love you , i love you <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> May I", max_len=20)

'<start> may i get witcha can i get witcha <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i need", max_len=20)

'<start> i need a little guidance <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> what should", max_len=20)

'<start> what should i do <end> '

# 6. 회고

- 예제에서 사용했던 모델의 layer 는 그대로 사용하였다. 초기 모델의 성능이 꽤나 괜찮았기 때문이다. 목표로 했던 10epoch 이내 val_loss 2.2 이하(2.04)가 달성되었고, 생성결과의 의미를 이해할 수 있었다. 
- 파리미터 조정 외에 모델의 성능을 올리기 위한 설계과정에서 embedding size 나 hidden size 가 변화함에 따라 결과가 어떨지 예상할 수 있는 insight 를 갖는 것이 중요할 것 같다. 
- 데이터 처리 과정에서 토큰화 과정을 세부적으로 더 깊게 봐야겠다는 생각이 들었다.
- 텍스트생성 결과를 여러가지로 테스트 해보았는데, 특수문자로 인한 문장겹침 현상이 보였지만, 의미전달이 가능한 문장이 생성되었다. 
- 각 벡터들의 시각화를 해보고 싶었는데 진행하지 못했다.(얼마나 복잡하게 그려질지 궁금했다)-> 경험자에 따르면 크게 의미없었다고 한다. 이유는 근접 벡터가 상호 관련성이 없기 떄문에 시각화를 통한 인사이트를 얻을 수 없기 때문이라고 한다.
